In [ ]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

ModuleNotFoundError: No module named 'yfinance'

## Group Assignment
### Team Number: XX
### Team Member Names: Nelson Kang, 
### Team Strategy Chosen: Market Beat

Goal: Market beat
Strategy: Sortino-centered stock selection with some short-term momentum tilt

Inputs:
- CSV file with tickers

Outputs:
- 

In [ ]:
#Imports
import math
import time
import random
import numpy as np
import pandas as pd
import yfinance as yf
import os


# Defining most constants
N_min = 10 # min number of stocks in portfolio
N_max = 25 # max number of stocks in portfolio
max_weight = 0.15 # max weight per stock
sector_cap = 0.4 # max weight per sector
budget = 1_000_000 
MIN_avg_daily_volume = 5000
LIVE_date = "2025-11-21"
END_date = "2025-11-28"

fee_flat_usd = 2.15
fee_per_share_usd = 0.001




In [15]:

tk = yf.Ticker("AAPL")
print(tk)

yfinance.Ticker object <AAPL>


In [ ]:
#HELPERS
def get_tickers(file_name):
    """
    Input: local name of tickers csv file
    Output: list of tickers
    """
    df = pd.read_csv(file_name)
    first_col = df.columns[0]
    tickers = (df[first_col].str.strip().tolist())
    return tickers

def get_info(tickers_lst):
    """
    Input: list of tickers
    Output: dict: {ticker: {'sector': str, 'marketCap': float, 'currency': str}} 
    Function fetches basic info for list of tickers
    """
    output = {}
    for ticker in tickers_lst:
        try:
            t = yf.Ticker(ticker)
            info = t.get_info()
            sector = info.get("industry")
            mcap = info.get("marketCap")
            currency = info.get("currency")
            output[t] = {
                        'ticker' : ticker,
                        'sector' : sector,
                         'MarketCap' : mcap,
                         'Currency' : currency}
        except Exception as e:
            output[t] = {'ticker' : None,
                        'sector' : None,
                         'MarketCap' : None,
                         'Currency' : None}
    return output

def get_exchange_rate():
    """
    Input: None
    Output: Float
    Returns latest exchange rate CAD --> USD
    """
    fx = yf.Ticker("CADUSD=X")
    return float(fx.fast_info["last_price"])


def weekly_closing(ticker_lst, start, end):
    full_data = yf.download(ticker_lst, start, end, progress=False, auto_adjust=True)["Close"]
    return full_data.resample("W").last()


def get_history(ticker_lst, start, end):
    """
    
    """
    data = yf.download(ticker_lst, start, end, interval='1d', auto_adjust=False, progress=False)
    prices = data['Adj Close'].copy()
    vols = data["Volume"].copy()
    return prices, vols

def monthly_filter_avg_volume(vol_series):
    return 

In [ ]:
#PORTFOLIO CONSTRUCTION
def possible_companies(ticker_lst, FX):

    all_prices, Volume_all = get_history(ticker_lst, "2024-10-01", "2025-09-30")

    pos_companies = {}

    for t in ticker_lst:
        try: # filter out ellegible stocks
            # check for average daily volume to meet rule requirements
            vol_ser = Volume_all.get(t)
            avg_volume = monthly_filter_avg_volume(vol_ser)
            last_close = float()
            if avg_volume < MIN_avg_daily_volume:
                continue


            t_obj = yf.Ticker(t)   
            info = t_obj.get_info()
            sector = info.get("industry")
            mcap = info.get("marketCap")
            currency = info.get("currency")
            last_close = float(t_obj.fast_info["last_price"])

            pos_companies[t] = {
                'Ticker' : t,
                'Currency' : currency,
                'Sector' : sector,
                'Market Cap (cad)' : mcap,
                'Last Price' : last_close,
            }

        except Exception:
            continue # in case a step errors

    return pos_companies


In [ ]:
def build_port(ticker_lst):
    FX = get_exchange_rate()

    possible_tickers = possible_companies(ticker_lst, FX)
    return


#tech_data =yf.download(ticker_list, start, end, progress=False, auto_adjust=True)["Close"]

t_list = get_tickers("Tickers_file.csv")
print(t_list)
#info_dict = get_info(t_list)
#df1 = pd.DataFrame(info_dict)
#weekly_df = weekly_closing(t_list, "2022-01-01", "2025-08-08")
prices_all, vols_all = get_history(t_list, "2022-01-01", "2025-08-08")
print(vols_all)
print(prices_all)
ex = get_exchange_rate()
print(ex)


['ABBV', 'ABT', 'ACN', 'AGN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BB.TO', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CELG', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MON', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTN', 'RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']



4 Failed downloads:
['RTN', 'CELG', 'AGN', 'MON']: YFTzMissingError('possibly delisted; no timezone found')


Ticker            ABBV         ABT         ACN  AGN        AIG        AMZN  \
Date                                                                         
2022-01-03  116.779297  128.996155  383.711884  NaN  52.754250  170.404495   
2022-01-04  116.555092  125.962349  380.969849  NaN  54.234207  167.522003   
2022-01-05  117.167358  125.396385  374.260712  NaN  53.489620  164.356995   
2022-01-06  116.615440  125.377838  356.187500  NaN  54.408844  163.253998   
2022-01-07  116.313637  125.767532  349.355865  NaN  55.833649  162.554001   
...                ...         ...         ...  ...        ...         ...   
2025-08-01  193.822708  126.726479  255.369995  NaN  76.550179  214.750000   
2025-08-04  195.778610  129.354782  258.779999  NaN  77.832642  211.649994   
2025-08-05  197.128876  130.111435  247.070007  NaN  77.981766  213.750000   
2025-08-06  194.894989  130.350357  247.539993  NaN  78.598145  222.309998   
2025-08-07  197.436661  131.475357  241.720001  NaN  76.152512  

In [ ]:
for n in range(10, 25):
    min_weight = 100/(2*n)

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Nelson Kang
